In [ ]:
import torch
from torchvision import datasets, transforms

# Daten laden und präparieren

In diesem Beispiel sehen wir uns MNIST an.
Hierbei handelt es sich um eine Datenbank aus
70000 handgeschriebenen Ziffern, davon 60000 im Trainings-
und 10000 im Testdatensatz.

In [ ]:
train_set = datasets.MNIST("data", train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor()]))
test_set = datasets.MNIST("data", train=False, download=True,
                       transform=transforms.Compose([transforms.ToTensor()]))

In [ ]:
train_set

Diesen Datensatz packen wir nun in einen passenden `DataLoader`.
Mit diesem können wir gut über Datensätze iterieren.

In [ ]:
batch_size = 50

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

Das erste Element des DataLoaders sieht wie folgt aus:

In [ ]:
fst_data = next(iter(train_loader))
print(fst_data)

Betrachten beispielhaft wir die erste Ziffer.

In [ ]:
fst_image = fst_data[0][0]
fst_digit = fst_data[1][0].item()

import matplotlib.pyplot as plt
plt.imshow(fst_image.view(fst_image.shape[1], fst_image.shape[2]))
plt.title(f"Value: {fst_digit}")

# Neuronales Netz erstellen

### Multilayer Perceptron
Nun erstellen wir die Architektur des neuronalen Netzes. Wir verwenden hier ein Fully Connected Neural Network mit einem Hidden Layer mit 64 Knoten.

<img src="assets/fcnn-28x28-64-10.svg"/>

In [ ]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(28*28, 64)
        self.fc2 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.log_softmax(self.fc2(x), dim=1)
        return x

In [ ]:
lr_rate = 0.01

In [ ]:
model = MultiLayerPerceptron()
model

# Neuronales Netz trainieren

Jetzt initialisieren wir die Loss class und den Optimizer (Stochastic Gradient Descent).

In [ ]:
loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
def calculate_accuracy(loader):
    correct = 0
    total = 0
    for images_test, labels_test in loader:
        images_test = Variable(images_test.view(-1, 28*28))
        outputs = model(images_test)
        _, predicted = torch.max(outputs.data, 1)
        total += labels_test.size(0)
        correct += (predicted == labels_test).sum()
    acc = correct.item() / total
    return acc

In [ ]:
from torch.autograd import Variable

epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for images, labels in train_loader:
        # we use torch.autograd.Variable for backpropagation
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    test_accuracy = calculate_accuracy(test_loader)
    print(f"Loss: {loss.item():.5f}. Test Accuracy: {test_accuracy:.3f}.")